
# Run the trained model on Android Device Via TensorFlow



Pytorch is very usefull for developing and testing model and for research purposes but a minor downside of PyTorch is that it is not very usefull for deployment of the AI Models so therefore we will use Google's Tensorflow to deploy model on Android Device so we will now have to convert the model to tensorflow so that we can deploy that to Android

## Convert from PyTorch to ONNX
ONNX (Open Neural Network Exchange) is open format for exchange of Deep Learning model between different Frameworks. In other words it is format which is widely used for exchanging one model from one framework to other framework. We will first convert model from Pytorch to ONNX so we can transfer it to another framework.   
The Process of converting to ONNX is that we make a random dummy input and pass it through the model in pytorch and then pass it  throught the function of onnx export to export it to a .onnx file. The shape of the dummy input should be (1, shape of single images) in detail the input would be (1, number of color channels, width of picture , height of picture )`


In [1]:
# helper functions for this notebook
def gvd(variable):
    print("The data type of this variable is " + str(type(variable)))
import time
def mills():
    return int(round(time.time() * 1000))

### Loading Pytorch Model

In [2]:
!pip list | grep 'torch'

torch                         1.7.0+cu101    
torchsummary                  1.5.1          
torchtext                     0.3.1          
torchvision                   0.8.1+cu101    


In [3]:
import torch
import torchvision
import torch.nn as nn

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load model from torchvision model zoo

In [5]:
# You can load model from torchvision models or below you can load local model file.
model = torchvision.models.mobilenet_v2(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


OR  


**Load model from local file**

for more help GOTO: https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [7]:
from torch import nn
import torch.onnx

class Generator(nn.Module):
    def __init__(self):

        super(Generator, self).__init__()

        self.conv1_1 = torch.nn.Conv2d(3, 8, kernel_size=3, padding=1).to(dtype=torch.float)
        self.bn1_1 = torch.nn.BatchNorm2d(8).to(dtype=torch.float)
        self.relu1_1 = torch.nn.ReLU()

        self.conv2_1 = torch.nn.Conv2d(8, 8, kernel_size=3, padding=1).to(dtype=torch.float)
        self.bn2_1 = torch.nn.BatchNorm2d(8).to(dtype=torch.float)
        self.relu2_1 = torch.nn.ReLU()

        self.conv3_1 = torch.nn.Conv2d(8, 32, kernel_size=3, padding=1).to(dtype=torch.float)
        self.relu3_1 = torch.nn.ReLU()
        self.pixel_shuffle3_1 = torch.nn.PixelShuffle(2)

        self.conv4_1 = torch.nn.Conv2d(8, 32, kernel_size=3, padding=1).to(dtype=torch.float)
        self.relu4_1 = torch.nn.ReLU()
        self.pixel_shuffle4_1 = torch.nn.PixelShuffle(2)

        self.conv5_1 = torch.nn.Conv2d(8, 32, kernel_size=3, padding=1).to(dtype=torch.float)
        self.relu5_1 = torch.nn.ReLU()
        self.pixel_shuffle5_1 = torch.nn.PixelShuffle(2)

        self.conv6_1 = torch.nn.Conv2d(8, 3, kernel_size=3, padding=1).to(dtype=torch.float)

    def forward(self, x):
        x = self.conv1_1(x)
        x = self.bn1_1(x)
        x = self.relu1_1(x)

        x = self.conv2_1(x)
        x = self.bn2_1(x)
        x = self.relu2_1(x)

        x = self.conv3_1(x)
        x = self.relu3_1(x)
        x = self.pixel_shuffle3_1(x)

        x = self.conv4_1(x)
        x = self.relu4_1(x)
        x = self.pixel_shuffle4_1(x)

        x = self.conv5_1(x)
        x = self.relu5_1(x)
        x = self.pixel_shuffle5_1(x)

        x = self.conv6_1(x)
        
        out = (torch.tanh(x) + 1) / 2

        return out

torch_model = Generator()
torch_model.cuda()
torch_model.load_state_dict(torch.load("/content/drive/My Drive/data/epochs/netG_epoch_8_90.pth"))
torch_model.eval()

Generator(
  (conv1_1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1_1): ReLU()
  (conv2_1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2_1): ReLU()
  (conv3_1): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3_1): ReLU()
  (pixel_shuffle3_1): PixelShuffle(upscale_factor=2)
  (conv4_1): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4_1): ReLU()
  (pixel_shuffle4_1): PixelShuffle(upscale_factor=2)
  (conv5_1): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5_1): ReLU()
  (pixel_shuffle5_1): PixelShuffle(upscale_factor=2)
  (conv6_1): Conv2d(8, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

### Perform simple PyTorch inference

In [30]:
image_size = 256 # size of input image
color_channels = 3
batch_size = 1
dummy_input = torch.rand(batch_size, color_channels, image_size, image_size)
print(dummy_input.size())
print((dummy_input.cuda()).size())

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


In [32]:
# performs inference with pytorch
start_time = mills()
output = torch_model(dummy_input.cuda())
end_time = mills()
print("model single inference in milliSeconds ", end_time - start_time)
print("model output shape: ", output.shape)

model single inference in milliSeconds  3
model output shape:  torch.Size([1, 3, 2048, 2048])


## Convert to ONNX model

In [40]:
# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                              dummy_input.cuda(),                       # model input (or a tuple for multiple inputs)
                              'eye_state_model_ONNX.onnx', # where to save the model (can be a file or file-like object)
                              export_params=True,       # store the trained parameter weights inside the model file
                              input_names=['main_input'],  # specify the name of input layer in onnx model
                              output_names=['main_output'])     # specify the name of input layer in onnx model

### Run ONNX model with ONNX runtime

OnnxRuntime is used for running ONNX model

In [35]:
# install onnxruntime
!pip install --upgrade onnxruntime

     |████████████████████████████████| 3.8MB 4.8MB/s 


In [39]:
import onnxruntime as nxrun

sess = nxrun.InferenceSession('/content/eye_state_model_ONNX.onnx')
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
start_time = mills()
# run onnx model with onnx runtime python
result = sess.run(None, {input_name: dummy_input.numpy()})

end_time = mills()
print("model single inference in milliSeconds on onnxruntime", end_time - start_time)
print("Output", result)

model single inference in milliSeconds on onnxruntime 666
Output [array([[[[0.43348402, 0.4199896 , 0.41498637, ..., 0.35747483,
          0.31060505, 0.3940094 ],
         [0.39806074, 0.40101653, 0.42159224, ..., 0.28493536,
          0.2831241 , 0.32416904],
         [0.3988735 , 0.4192746 , 0.39254513, ..., 0.24770582,
          0.24658152, 0.30567044],
         ...,
         [0.40748838, 0.42893073, 0.45143926, ..., 0.5031651 ,
          0.47874892, 0.46750182],
         [0.39659277, 0.4151328 , 0.43215007, ..., 0.4823483 ,
          0.46379876, 0.45471627],
         [0.38823447, 0.37931353, 0.42069522, ..., 0.45294517,
          0.44677478, 0.44986475]],

        [[0.45000586, 0.44163433, 0.43215123, ..., 0.33709872,
          0.34361   , 0.3636375 ],
         [0.42680627, 0.43772098, 0.43013808, ..., 0.3504155 ,
          0.32852006, 0.36744022],
         [0.4416224 , 0.42279786, 0.4027039 , ..., 0.28841937,
          0.28787106, 0.3319966 ],
         ...,
         [0.50858176, 

## Convert from ONNX to TensorFlow FreezeGraph


**We will use onnx-tf to convert model**  

onnx_tf is the library build by onnx team which is used to transfer the model from onnx to tensorflow it can create a backend to enable the model to run with tensorflow. We will first load the saved .onnx model file with onnx.load and the by using prepare function of onnx_tf prepare that loaded model to be run by tensorflow. and by using the export_graph function of that prepared backend we can export this model in a file format with .tf extension supported by Original Tensorflow

In [41]:
# install onnx and onnx-tf
!pip install --upgrade onnx | tail -n 2
!pip install  onnx-tf==1.5.0 | tail -n 2

In [42]:
# Tell google colab to use tensorflow version 1.x
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [43]:
# see versions of all tensorflow libraries installed
!pip list | grep 'flow'

mesh-tensorflow               0.1.12         
tensorflow                    1.15.2         
tensorflow-addons             0.8.3          
tensorflow-datasets           4.0.1          
tensorflow-estimator          1.15.1         
tensorflow-gan                2.0.0          
tensorflow-gcs-config         2.3.0          
tensorflow-hub                0.10.0         
tensorflow-metadata           0.24.0         
tensorflow-privacy            0.2.2          
tensorflow-probability        0.7.0          


In [44]:
import onnx
import tensorflow as tf

In [45]:
# Either convert onnx to tensorflow protobuf model by using command
# -i defined input model -o defines output model name
!onnx-tf convert -i "/content/eye_state_model_ONNX.onnx" -o  '/content/eye_state_model_tensorFlow.pb' 

/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.









/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP,

**OR**
Code in next block can also be used to convert onnx to tf freezegraph  
**It is not tested by me**

In [47]:
# load the model saved in onnx format
from onnx_tf.backend import prepare

model_onnx = onnx.load('eye_state_model_ONNX.onnx')
onnx.checker.check_model(model_onnx)

# prepare model for exporting to tensorFlow using tensorFlow backend
tf_rep = prepare(model_onnx)
start_time = mills()
print(tf_rep.run(dummy_input))
end_time = mills()
print(end_time - start_time)
print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

# # export tensorFlow backend to tensorflow tf file
tf_rep.export_graph('eye_state_model_tensorFlow.pb')
tf_rep.run(dummy_input)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


/usr/local/lib/python3.6/dist-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/usr/local/lib/python3.6/dist-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version 

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Outputs(main_output=array([[[[0.433484  , 0.41998953, 0.4149863 , ..., 0.3574749 ,
          0.31060505, 0.3940094 ],
         [0.39806068, 0.40101647, 0.42159218, ..., 0.2849354 ,
          0.28312412, 0.32416907],
         [0.39887345, 0.4192745 , 0.39254504, ..., 0.24770588,
          0.24658155, 0.30567044],
         ...,
         [0.40748835, 0.4289307 , 0.45143926, ..., 0.5031651 ,
          0.4787489 , 0.46750182],
         [0.39659274, 0.4151328 , 0.43215   , ..., 0.4823483 ,
          0.46379876, 0.45471627],
         [0.38823444, 0.37931353, 0.4206952 , ..., 0.45294517,
          0.44677478, 0.44986472]],

        [[0.4500058 , 0.44163424, 0.43215117, ..., 0.33709878,
          0.34361005, 0.3636375 ],
         [0.4268062 , 0.43772087, 0.430138  , ..., 0.35041553,
          0.32852006, 0.36744022],
         [0.44162235, 0.42279774, 0.40270382, ..., 0.28841946,
          0.28787112, 0.33199662],
  

Outputs(main_output=array([[[[0.433484  , 0.41998953, 0.4149863 , ..., 0.3574749 ,
          0.31060505, 0.3940094 ],
         [0.39806068, 0.40101647, 0.42159218, ..., 0.2849354 ,
          0.28312412, 0.32416907],
         [0.39887345, 0.4192745 , 0.39254504, ..., 0.24770588,
          0.24658155, 0.30567044],
         ...,
         [0.40748835, 0.4289307 , 0.45143926, ..., 0.5031651 ,
          0.4787489 , 0.46750182],
         [0.39659274, 0.4151328 , 0.43215   , ..., 0.4823483 ,
          0.46379876, 0.45471627],
         [0.38823444, 0.37931353, 0.4206952 , ..., 0.45294517,
          0.44677478, 0.44986472]],

        [[0.4500058 , 0.44163424, 0.43215117, ..., 0.33709878,
          0.34361005, 0.3636375 ],
         [0.4268062 , 0.43772087, 0.430138  , ..., 0.35041553,
          0.32852006, 0.36744022],
         [0.44162235, 0.42279774, 0.40270382, ..., 0.28841946,
          0.28787112, 0.33199662],
         ...,
         [0.50858176, 0.5370097 , 0.5129028 , ..., 0.65669465,
     

List all Operations in a model

In [ ]:
# export_dir = './saved'
# graph_pb = '/content/eye_state_model_tensorFlow.pb'

# builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

# with tf.gfile.GFile(graph_pb, "rb") as f:
#     graph_def = tf.GraphDef()
#     graph_def.ParseFromString(f.read())
#     [n.name for n in tf.get_default_graph().as_graph_def().node]

# for i in tf.get_default_graph().get_operations():
#     print(i)
# print(tf.get_default_graph())


In [ ]:
# # from PIL import Image
# # print('Image 1:')
# # img = Image.open('/content/dataset_facialImages/test/CloseFace/closed_eye_0089.jpg_face_1.jpg')
# # display(dummy_input)
# # print(np.asarray(img, dtype=np.float32)[np.newaxis, np.newaxis, :, :].shape)
# with tf.device('/CPU:0'):
#     for e in 'abcde':
#         output = tf_rep.run(dummy_input)
# for e in 'abcde':
#     print(model(dummy_input))
# print('The digit is classified as ', output)#np.argmax(output))

## Convert the model to TensorFlow Lite
Tensorflow Lite is designed to run on mobiles and other embedded systems with limited computing power and resources it do this by converting the model to specific format and introducing new types of optimization techniques. So we must have to save the model to those specific formats.  
TensorFlow saves the model in reduced file size and use different optimization techniques by not changing the accuracy. Optionaly we can more increase the speed of model with some trade offs but this is optional so we will tell about it in future sections  
####**TensorFlow Lite Converter**
This converter is the Python API which convert the TensorFlow Model to TensorFlow Lite Format it also introduce some optimization techniques which we will follow in coming Sections

In [ ]:
import tensorflow as tf

In [48]:
import tensorflow as tf
# make a converter object from the saved tensorflow file
converter = tf.lite.TFLiteConverter.from_frozen_graph('/content/eye_state_model_tensorFlow.pb', #TensorFlow freezegraph .pb model file
                                                      input_arrays=['main_input'], # name of input arrays as defined in torch.onnx.export function before.
                                                      output_arrays=['main_output']  # name of output arrays defined in torch.onnx.export function before.
                                                      )
# tell converter which type of optimization techniques to use
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# to view the best option for optimization read documentation of tflite about optimization
# go to this link https://www.tensorflow.org/lite/guide/get_started#4_optimize_your_model_optional

# convert the model 
tf_lite_model = converter.convert()
# save the converted model 
open('/content/eye_state_model_tensorFlow.tflite', 'wb').write(tf_lite_model)

ConverterError: ignored

## Inference of TensorFlow Lite model with Python

In [ ]:
import numpy as np
import tensorflow as tf
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/eye_state_model_tensorFlow.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
print(input_details)
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = dummy_input.numpy()
interpreter.set_tensor(input_details[0]['index'], input_data)
start_time = mills()

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
end_time = mills()
print("Time taken to one inference in milliseconds", end_time - start_time)
print("output of model",output_data)

Below blocks are used to run inference on single image

In [ ]:
import torch
from torchvision import transforms
transform = transforms.Compose([
        transforms.CenterCrop(2256), # center crop every image to 224 pixels
        transforms.ToTensor(), # convert the image to Tensor
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))                       
])


In [ ]:
f = tensor.view(1,224,224,3 )
print(f.shape)

In [ ]:
from PIL import Image
img = Image.open('/content/obama.jpeg')
img = img.convert('RGB')
img.save('/content/tst3.jpg')
tensor = transform(img)

# if gpu_available:
#     tensor = tensor.cuda()

print(tensor.shape)
tensor.unsqueeze_(0)
print(tensor.shape)
# print(model(tensor))
dummy_input = tensor

In [ ]:
dummy_input.shape

In [ ]:
img.save('/content/benazir.png')


In [ ]:
model = model.cpu()
torch.save(model.state_dict(), 'state_dict.pt')

In [ ]:
del model

In [ ]:
model = torchvision.models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=2)

In [ ]:
# vgg19
model = torchvision.models.vgg19(pretrained=True)
model.classifier[6] = model.classifier[6] = nn.Linear(4096,2)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [ ]:
model.load_state_dict(torch.load('state_dict.pt', map_location='cpu'))


<All keys matched successfully>

In [ ]:
!git clone https://github.com/tensorflow/examples.git

Cloning into 'examples'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 10440 (delta 55), reused 90 (delta 15), pack-reused 10298
Receiving objects: 100% (10440/10440), 18.76 MiB | 21.63 MiB/s, done.
Resolving deltas: 100% (5389/5389), done.


In [ ]:
!zip -r f.zip /content/examples/lite/examples/image_classification/android/images/

updating: content/examples/lite/examples/image_classification/android/images/ (stored 0%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img5.png (deflated 2%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img8.png (deflated 6%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img7.png (deflated 5%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img2.png (deflated 5%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img1.png (deflated 10%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img4.png (deflated 1%)
  adding: content/examples/lite/examples/image_classification/android/images/classifydemo_img6.png (deflated 14%)
